# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [97]:
df = df.dropna()
df = df.drop(['Unnamed: 0','data_ref','id_cliente'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   12427 non-null  object 
 1   posse_de_veiculo       12427 non-null  bool   
 2   posse_de_imovel        12427 non-null  bool   
 3   qtd_filhos             12427 non-null  int64  
 4   tipo_renda             12427 non-null  object 
 5   educacao               12427 non-null  object 
 6   estado_civil           12427 non-null  object 
 7   tipo_residencia        12427 non-null  object 
 8   idade                  12427 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  12427 non-null  float64
 11  renda                  12427 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.1+ MB


In [98]:
train, test = train_test_split(df, test_size=0.25, random_state=100)

2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [99]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil) + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego + qt_pessoas_residencia'
md = smf.ols(modelo, data = test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.01 #Usar o parâmetro zerado dá erro de implementação
                         , alpha = 0.1)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     53.80
Date:                Wed, 01 Mar 2023   Prob (F-statistic):          1.76e-220
Time:                        19:18:19   Log-Likelihood:                -31733.
No. Observations:                3107   AIC:                         6.352e+04
Df Residuals:                    3082   BIC:                         6.367e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                         -6716.4139   9420.672     -0.713      0.476   -2.52e+04    1.18e+04
sexo[T.M]                                          5881.0853    269.774     21.800      0.000    5352.129    6410.041
posse_de_veiculo[T.True]                           -157.9320    259.320     -0.609      0.543    -666.390     350.525
posse_de_imovel[T.True]                             564.1797    260.452      2.166      0.030      53.503    1074.857
C(tipo_renda)[T.Bolsista]                         -1395.8199   2979.583     -0.468      0.639   -7237.990    4446.350
C(tipo_renda)[T.Empresário]                        1096.5286    274.523      3.994      0.000     558.261    1634.796
C(tipo_renda)[T.Pensionista]                      -5366.1233   6641.854     -0.808      0.419   -1.84e+04    7656.786
C(tipo_renda)[T.Servidor público]                   768.8602    408.050      1.884      0.060     -31.218    1568.938
C(educacao, Treatment(2))[T.Primário]               803.4307   1427.053      0.563      0.573   -1994.640    3601.502
C(educacao, Treatment(2))[T.Pós graduação]         -321.6945   3325.796     -0.097      0.923   -6842.695    6199.306
C(educacao, Treatment(2))[T.Superior completo]     1213.8937    254.635      4.767      0.000     714.621    1713.166
C(educacao, Treatment(2))[T.Superior incompleto]   -831.9017    623.846     -1.334      0.182   -2055.099     391.295
C(estado_civil)[T.Separado]                        1492.1519   4732.197      0.315      0.753   -7786.428    1.08e+04
C(estado_civil)[T.Solteiro]                        1396.1960   4688.761      0.298      0.766   -7797.216    1.06e+04
C(estado_civil)[T.União]                           -751.9776    474.620     -1.584      0.113   -1682.582     178.627
C(estado_civil)[T.Viúvo]                           1239.4640   4760.711      0.260      0.795   -8095.024    1.06e+04
C(tipo_residencia, Treatment(1))[T.Aluguel]         -27.2922    982.200     -0.028      0.978   -1953.125    1898.541
C(tipo_residencia, Treatment(1))[T.Com os pais]    -315.6132    555.959     -0.568      0.570   -1405.700     774.474
C(tipo_residencia, Treatment(1))[T.Comunitário]    -209.2722   1578.446     -0.133      0.895   -3304.185    2885.641
C(tipo_residencia, Treatment(1))[T.Estúdio]        -513.9941   1364.883     -0.377      0.707   -3190.166    2162.178
C(tipo_residencia, Treatment(1))[T.Governamental]  1111.1058    716.256      1.551      0.121    -293.281    2515.493
qtd_filhos                                        -1019.5787   4709.499     -0.216      0.829   -1.03e+04    8214.496
idade                                                49.7886     14.965      3.327      0.001      20.446      79.132
tempo_emprego  

R2 para alphas:
- 0: 0.304
- 0.001: 0.304
- 0.005: 0.304
- 0.01: 0.304
- 0.05: 0.304
- 0.1: 0.304

Considerando apenas os ajustes de alfa propostos não há melhor modelo, visto que não se percebe alterações relevantes no modelo para o R2, mas também não se alteram R2 ajustado e AIC. Isso indica que as penalizações alfas são irrisórias para os valores propostos.

3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [100]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil) + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego + qt_pessoas_residencia'
md = smf.ols(modelo, data = test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.1)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     53.80
Date:                Wed, 01 Mar 2023   Prob (F-statistic):          1.76e-220
Time:                        19:18:21   Log-Likelihood:                -31733.
No. Observations:                3107   AIC:                         6.352e+04
Df Residuals:                    3082   BIC:                         6.367e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                         -6716.4139   9420.672     -0.713      0.476   -2.52e+04    1.18e+04
sexo[T.M]                                          5881.0853    269.774     21.800      0.000    5352.129    6410.041
posse_de_veiculo[T.True]                           -157.9320    259.320     -0.609      0.543    -666.390     350.525
posse_de_imovel[T.True]                             564.1797    260.452      2.166      0.030      53.503    1074.857
C(tipo_renda)[T.Bolsista]                         -1395.8199   2979.583     -0.468      0.639   -7237.990    4446.350
C(tipo_renda)[T.Empresário]                        1096.5286    274.523      3.994      0.000     558.261    1634.796
C(tipo_renda)[T.Pensionista]                      -5366.1233   6641.854     -0.808      0.419   -1.84e+04    7656.786
C(tipo_renda)[T.Servidor público]                   768.8602    408.050      1.884      0.060     -31.218    1568.938
C(educacao, Treatment(2))[T.Primário]               803.4307   1427.053      0.563      0.573   -1994.640    3601.502
C(educacao, Treatment(2))[T.Pós graduação]         -321.6945   3325.796     -0.097      0.923   -6842.695    6199.306
C(educacao, Treatment(2))[T.Superior completo]     1213.8937    254.635      4.767      0.000     714.621    1713.166
C(educacao, Treatment(2))[T.Superior incompleto]   -831.9017    623.846     -1.334      0.182   -2055.099     391.295
C(estado_civil)[T.Separado]                        1492.1519   4732.197      0.315      0.753   -7786.428    1.08e+04
C(estado_civil)[T.Solteiro]                        1396.1960   4688.761      0.298      0.766   -7797.216    1.06e+04
C(estado_civil)[T.União]                           -751.9776    474.620     -1.584      0.113   -1682.582     178.627
C(estado_civil)[T.Viúvo]                           1239.4640   4760.711      0.260      0.795   -8095.024    1.06e+04
C(tipo_residencia, Treatment(1))[T.Aluguel]         -27.2922    982.200     -0.028      0.978   -1953.125    1898.541
C(tipo_residencia, Treatment(1))[T.Com os pais]    -315.6132    555.959     -0.568      0.570   -1405.700     774.474
C(tipo_residencia, Treatment(1))[T.Comunitário]    -209.2722   1578.446     -0.133      0.895   -3304.185    2885.641
C(tipo_residencia, Treatment(1))[T.Estúdio]        -513.9941   1364.883     -0.377      0.707   -3190.166    2162.178
C(tipo_residencia, Treatment(1))[T.Governamental]  1111.1058    716.256      1.551      0.121    -293.281    2515.493
qtd_filhos                                        -1019.5787   4709.499     -0.216      0.829   -1.03e+04    8214.496
idade                                                49.7886     14.965      3.327      0.001      20.446      79.132
tempo_emprego  

R2 para alphas:
- 0: 0.304
- 0.001: 0.304
- 0.005: 0.304
- 0.01: 0.304
- 0.05: 0.304
- 0.1: 0.304

Tal qual no modelo anterior, considerando apenas os ajustes de alfa propostos não há melhor modelo, visto que não se percebe alterações relevantes no modelo para o R2, mas também não se alteram R2 ajustado e AIC. Isso indica que as penalizações alfas são irrisórias para os valores propostos.

4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?

In [101]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            print(included+[new_column])
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [171]:
y = pd.DataFrame(test['renda'])
x = test.drop(['renda'],axis=1)
x = pd.get_dummies(x, columns=['sexo'], drop_first=True)
x = pd.get_dummies(x, columns=['tipo_renda'], drop_first=True)
x = pd.get_dummies(x, columns=['educacao'], drop_first=True)
x = pd.get_dummies(x, columns=['estado_civil'], drop_first=True)
x = pd.get_dummies(x, columns=['tipo_residencia'], drop_first=True)
x = pd.get_dummies(x, columns=['posse_de_veiculo'], drop_first=True)
x = pd.get_dummies(x, columns=['posse_de_imovel'], drop_first=True)
x = x.reset_index(drop=True)
y = y.reset_index(drop=True)

In [170]:
result = stepwise_selection(x.astype(float), y.astype(float))

['tipo_residencia_Comunitário']
['posse_de_imovel_True']
['tempo_emprego']
['tipo_renda_Empresário']
['sexo_M']
['estado_civil_Separado']
['tipo_renda_Bolsista']
['educacao_Superior incompleto']
['posse_de_veiculo_True']
['educacao_Superior completo']
['qtd_filhos']
['tipo_residencia_Com os pais']
['qt_pessoas_residencia']
['tipo_residencia_Casa']
['educacao_Pós graduação']
['idade']
['educacao_Secundário']
['tipo_renda_Pensionista']
['estado_civil_Solteiro']
['tipo_residencia_Estúdio']
['estado_civil_União']
['estado_civil_Viúvo']
['tipo_residencia_Governamental']
['tipo_renda_Servidor público']
Add                               2 with p-value 7.51872e-125
[2]


C:\Users\Acer\AppData\Local\Temp\ipykernel_1272\661674484.py:24: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


KeyError: "None of [Int64Index([2], dtype='int64')] are in the [columns]"

5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.